In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import camb

plt.rcParams.update({"figure.figsize": (12, 8), "font.size": 14})

In [ ]:
# random number generator with a fixed seed
rng = np.random.default_rng(12345)

### Get power spectrum

In [ ]:
# cosmological parameters used to generate the power spectrum
cparams = camb.CAMBparams(ombh2=0.05 * 0.67**2, omch2=0.25 * 0.67**2, H0=67.)

# interpolation function for the matter power spectrum
pk = camb.get_matter_power_interpolator(cparams, nonlinear=False, zmax=1000)

In [ ]:
# Define a k axis to make some plots
# CAMB uses units of Mpc/h
kh = np.logspace(-4, 1, 1000)

In [ ]:
# Plot the power spectrum at a few redshifts
for z in [100, 10, 1, 0]:
    plt.plot(kh, pk.P(z, kh), label=f"z={z}")
plt.legend()
plt.loglog()
plt.xlabel("$k/h$ (h/Mpc)")
plt.ylabel("$P(k)$ $(Mpc/h)^3$")

### Draw a realisation of density contrast
We will draw random gaussian fluctuations from the power spectrum. We'll do this in 2d to simplify, so we need to scale the amplitude of the 3d power spectrum

In [ ]:
# Define a discrete grid
N = 512
k = np.arange(N)  # positive k
k_full = np.arange(2 * N) - N  # positive and negative k

# This is the redshift for the "early time" power spectrum
z0 = 1000

# Scale the integer grid to physical units to evaluate the power spectrum
# The power spectrum is isotropic, i.e. it depends only on the magnitude of k
# We only need to sample the half-space, because the density contrast is a
# real quantity so the negative frequencies are redundant
kmax = 4
k_grid = np.sqrt(k**2 + k_full[:, np.newaxis]**2) * kmax / N

# Draw random gaussian samples with zero mean and unit standard deviation
delta_k = 0.5**0.5 * (rng.standard_normal(size=(2 * N, N)) + 1j * rng.standard_normal(size=(2 * N, N)))

# Scale by the power spectrum to introduce k-dependence
delta_k *= np.sqrt(pk.P(z0, k_grid) * (2 * np.pi)**2 * k_grid / np.pi)
delta_k[k_grid == 0] *= 0  # ensure zero-mean

In [ ]:
# quick plot of k and delta^2(k)
fig, ax = plt.subplots(1, 2)
im = ax[0].imshow(k_grid, origin="lower")
plt.colorbar(im, ax=ax[0], label="k (h/Mpc)")
im = ax[1].imshow(np.log10(np.abs(delta_k)**2), origin="lower")
plt.colorbar(im, ax=ax[1], label="$\log_{10}\delta^2(k)$")

In [ ]:
# 2-d Fourier transform the samples to obtain the density contrast field
# We need to shift the order of the full axis to match the FFT convention
# which puts the positive frequencies first
delta = np.fft.irfft2(np.fft.fftshift(delta_k, axes=0))

In [ ]:
# Calculate the physical spatial axis after the Fourier transform
# We again shift the order to get back increasing x
x = np.fft.fftshift(np.fft.fftfreq(2 * N, kmax / N / (2 * np.pi)))

In [ ]:
# Plot the density contrast field
plt.imshow(delta, cmap="coolwarm", origin="lower", extent=(x.min(), x.max(), x.min(), x.max()))
plt.colorbar(label="$\delta$")
plt.xlabel("x (Mpc/h)")
plt.ylabel("y (Mpc/h)")

# zoom in if desired
#plt.xlim(-100, 100)
#plt.ylim(-100, 100)